We organized the ISLES Challenge in the blue_isc5935/share/team02/data directory (in the HiPerGator environment) into masks, NIFTIs, and processedNIFTIs directories. The NIFTIs directory has 3 NIFTI files for each subject, one NIFTI for each MRI modality. Originally, these files were in folders for each subject, along with some .json files. I moved the .nii files so that they were all in the same directory, making it easy to load those files in all at once. There may be a need to differentiate these files based on modality, in which case the file names can be checked. Similarly, the masks were placed in individual directories for each subject along with some other derivative information, so I isolated all those masks files into one directory, thus making it easy to load in that data if necessary. In anticpation of needing to remain organized after any preprocessing, I made an empty processedNIFTIs directory to store any future preprocessed files. ISLES only provides a training dataset, but we will likely split the existing training dataset into training, validation, and test sets, in which case we will use a split data function to do so. It seems that this can be done when loading in the data with relatively little complexity. Splitting up the data while loading can also allow us to change the training, validation, and test set proportions by using a function.

In [119]:
%load_ext autoreload
%autoreload 2

import torch
import torchvision
import matplotlib.pyplot as plt
import os
from os.path import join
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
import torchvision.transforms as transforms

import numpy as np

import itk

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Running on {device}")

import MyGenerator
      

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Running on cpu


In [19]:
root_path = "data/"
dataset = MyGenerator.MyDataset(join(join(root_path,'NIFTIs'),'NIFTIs').replace("\\","/"), join(root_path,'masks').replace("\\","/"))

In [108]:
def plot_batch(batch_imgs, batch_seg):
    batch_size = len(batch_imgs)
    fig, axs = plt.subplots(batch_size, 2, figsize=(10,5))
    print(batch_seg.shape)
    for i in range(batch_size):
        axs[i,0].imshow(batch_imgs[i,20,:,:], origin='lower')
        axs[i,0].set_title(f"Image {i} from batch")
        axs[i,1].imshow(batch_seg[i,20,:,:], origin='lower')
        axs[i,1].set_title(f"Seg {i} from batch")
        #axs[i].imshow(batch_imgs[i,10,:,:])
        #axs[i].set_title(f"Image {i} from batch")
    plt.show()

Running on cpu


In [120]:
def Question3(batchSize):
    myloader = DataLoader(dataset, batch_size=batchSize, shuffle=True)    

    ##
    # myiter = iter(myloader)
    # val  = next(myiter)
    for batch in myloader:
        x, y = batch
        #print("x: ", x.shape)
        #print("y: ", y.shape)
        plot_batch(x, y)
        print(type(x))
        print('Batch size:', x.size())
        break

In [123]:
import ipywidgets as widgets
from ipywidgets import interact, Layout

BatchSizeSlider = widgets.IntSlider(value=2,min=1,max=7,step=1,description="Batch Size")



#widgets.interact(FuncForWidget,HiddenLayers=HiddenLayerSlider,Neurons=NeuronsSlider,Epochs=EpochsSlider,ActHidden=ActHiddenDropDown,ActOutput=ActOutputDropDown)

button = widgets.Button(description='Start')

def on_button_click(button):
    output.clear_output()
    with output:
        Question3(BatchSizeSlider.value)

button.on_click(on_button_click)

output = widgets.Output()

display(BatchSizeSlider, button, output)

Running on cpu


IntSlider(value=2, description='Batch Size', max=7, min=1)

Button(description='Start', style=ButtonStyle())

Output()

In [36]:
#All cells below here were for testing and troubleshooting purposes

dataset.__getitem__(221)
JustANameImgItk, JustANameSegItk = dataset.__getitem__(3)
JustANameImgNP = np.asarray(JustANameImgItk)
JustANameSegNP = np.asarray(JustANameSegItk)
JustANameImgNP.shape

metadataIMG=itk.dict_from_image(JustANameImgItk)
metadataSEG=itk.dict_from_image(JustANameSegItk)



#metadataIMG
SEGpixdims = metadataSEG['spacing']
SEGdims = metadataIMG['size']

metadataIMG



batch_size = len(batch_imgs)
fig, axs = plt.subplots(batch_size, 2, figsize=(10,5))
print(batch_seg.shape)
for i in range(batch_size):
    axs[i,0].imshow(batch_imgs[i,31,:,:])
    axs[i,0].set_title(f"Image {i} from batch")
    axs[i,1].imshow(batch_seg[i,31,:,:])
    axs[i,1].set_title(f"Seg {i} from batch")
        #axs[i].imshow(batch_imgs[i,10,:,:])
        #axs[i].set_title(f"Image {i} from batch")
plt.show()

Running on cpu
(30, 192, 192)
(30, 192, 192)
sub-strokecase0222_ses-0001_adc.nii.gz
sub-strokecase0222_ses-0001_msk.nii.gz
NEW version
(72, 112, 112)
(72, 112, 112)
sub-strokecase0004_ses-0001_adc.nii.gz
sub-strokecase0004_ses-0001_msk.nii.gz
NEW version


RuntimeError: No suitable template parameter can be found.

In [44]:
case =47
#sub-strokecase0202_ses-0001_adc.nii.gz
file_name_msk = f"data/masks/sub-strokecase0{case:03d}_ses-0001_msk.nii.gz"
file_name_img = f"data/NIFTIs/NIFTIs/sub-strokecase0{case:03d}_ses-0001_adc.nii.gz"
mask = itk.imread(file_name_msk)
img = itk.imread(file_name_img)

print("Spacing:", itk.dict_from_image(img)['spacing'])

print(img.shape)
print(mask.shape)
itk.array_from_image(img)
itk.array_from_image(mask)

Spacing: (2.0, 2.0, 2.0)
(72, 112, 112)
(72, 112, 112)


array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [11]:
myloader

In [10]:
dataset.masks_names

['sub-strokecase0001_ses-0001_msk.nii.gz',
 'sub-strokecase0002_ses-0001_msk.nii.gz',
 'sub-strokecase0003_ses-0001_msk.nii.gz',
 'sub-strokecase0004_ses-0001_msk.nii.gz',
 'sub-strokecase0005_ses-0001_msk.nii.gz',
 'sub-strokecase0006_ses-0001_msk.nii.gz',
 'sub-strokecase0007_ses-0001_msk.nii.gz',
 'sub-strokecase0008_ses-0001_msk.nii.gz',
 'sub-strokecase0009_ses-0001_msk.nii.gz',
 'sub-strokecase0010_ses-0001_msk.nii.gz',
 'sub-strokecase0011_ses-0001_msk.nii.gz',
 'sub-strokecase0012_ses-0001_msk.nii.gz',
 'sub-strokecase0013_ses-0001_msk.nii.gz',
 'sub-strokecase0014_ses-0001_msk.nii.gz',
 'sub-strokecase0015_ses-0001_msk.nii.gz',
 'sub-strokecase0016_ses-0001_msk.nii.gz',
 'sub-strokecase0017_ses-0001_msk.nii.gz',
 'sub-strokecase0018_ses-0001_msk.nii.gz',
 'sub-strokecase0019_ses-0001_msk.nii.gz',
 'sub-strokecase0020_ses-0001_msk.nii.gz',
 'sub-strokecase0021_ses-0001_msk.nii.gz',
 'sub-strokecase0022_ses-0001_msk.nii.gz',
 'sub-strokecase0023_ses-0001_msk.nii.gz',
 'sub-strok